In [ ]:
using Polyhedra
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyDynamics
using StaticArrays
using RigidBodyTreeInspector
using JuMP
using Gurobi

In [ ]:
import MechanismComplementarity


In [ ]:
reload("MechanismComplementarity")
mc = MechanismComplementarity

In [ ]:
world = RigidBody{Float64}("world")
brick = Mechanism(world; gravity=SVector(0, 0, -9.81))

frame = CartesianFrame3D("dummy")
inertia = SpatialInertia(frame, 0 * eye(3), zeros(3), 0.0)
dummy = RigidBody(inertia)
base_x = Joint("base_x", Prismatic([1., 0, 0]))
attach!(brick, world, base_x, eye(Transform3D, frame_before(base_x), default_frame(world)), dummy)

frame = CartesianFrame3D("core")
inertia = SpatialInertia(frame, 0.1 * eye(3), zeros(3), 1.0)
core = RigidBody(inertia)
base_z = Joint("base_z", Prismatic([0., 0, 1]))
attach!(brick, dummy, base_z, eye(Transform3D, frame_before(base_z), default_frame(dummy)), core)

geoms = Dict(default_frame(core) => [HyperRectangle(Vec(-0.1, -0.1, -0.1), Vec(0.2, 0.2, 0.2))])



In [ ]:
env = mc.Environment(
    Dict(core => mc.ContactEnvironment(
            [Point3D(default_frame(core), SVector(0, 0, -0.1))],
            [
                mc.Obstacle(
                    default_frame(world),
                    SimpleHRepresentation{3, Float64}([0 0 1], [0]),
                    HalfSpace{3, Float64}([0, 0, 1], 0),
                    0.5
                ),
            ],
            [
                mc.FreeRegion(
                    default_frame(world),
                    SimpleHRepresentation{3, Float64}(
                        [0 0 -1],
                        [0])
                )
            ])
        )
)

limits = Dict{Joint, SimpleHRepresentation}()

In [ ]:
vis = Visualizer()[:brick]
setgeometry!(vis, brick, geoms)

In [ ]:
inspect(brick, vis)

In [ ]:
x0 = MechanismState(brick, [0., 0.5], [1.5, 0.0])

controller = x -> zeros(num_velocities(x))

Δt = 0.05
results = mc.simulate(x0, controller, limits, env, Δt, 30, GurobiSolver(OutputFlag=0));

In [ ]:
for r in results
    settransform!(vis, r.state)
    sleep(Δt)
end